# VGG

In [1]:
import os

# 设置代理
os.environ['http_proxy'] = 'http://127.0.0.1:7893'
os.environ['https_proxy'] = 'http://127.0.0.1:7893'
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7893'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7893'
os.environ['no_proxy'] = '127.0.0.1,localhost'
os.environ['NO_PROXY'] = '127.0.0.1,localhost'

# 验证代理设置
print(f"HTTP代理: {os.environ.get('http_proxy')}")

HTTP代理: http://127.0.0.1:7893


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time

In [ ]:
# 配置matplotlib - 适用于Linux服务器环境
import matplotlib
matplotlib.use('Agg')  # 使用非交互式后端，适合服务器环境
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Linux服务器中文字体配置
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

# 尝试使用系统中文字体，如果没有则使用默认字体
try:
    plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei', 'WenQuanYi Zen Hei', 'Droid Sans Fallback', 'SimHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
    print("中文字体配置完成")
except:
    print("使用默认字体")
    
# 清除matplotlib字体缓存
import matplotlib.font_manager as fm
fm._load_fontmanager(try_read_cache=False)

In [ ]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [ ]:
conv_arch = ((1,64), (1,128),(2,256), (2,512), (2,512))

In [ ]:
def vgg(conv_arch):
    conv_blocks = []
    in_channel = 1
    for (num_convs, out_channels) in conv_arch:
        conv_blocks.append(vgg_block(num_convs=num_convs, in_channels=in_channel, out_channels=out_channels))
        in_channel = out_channels
        
    return nn.Sequential(
        *conv_blocks,
        nn.Flatten(),
        nn.Linear(out_channels*7*7, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 10)
    )
    
net = vgg(conv_arch)

In [ ]:
X = torch.rand(1,1,224,224)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, " out shape:\t", X.shape)

Conv2d output shape:	 torch.Size([1, 48, 54, 54])
ReLU output shape:	 torch.Size([1, 48, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 48, 26, 26])
Conv2d output shape:	 torch.Size([1, 128, 26, 26])
ReLU output shape:	 torch.Size([1, 128, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 128, 12, 12])
Conv2d output shape:	 torch.Size([1, 177, 12, 12])
ReLU output shape:	 torch.Size([1, 177, 12, 12])
Conv2d output shape:	 torch.Size([1, 177, 12, 12])
ReLU output shape:	 torch.Size([1, 177, 12, 12])
Conv2d output shape:	 torch.Size([1, 128, 12, 12])
ReLU output shape:	 torch.Size([1, 128, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 128, 5, 5])
Flatten output shape:	 torch.Size([1, 3200])
Linear output shape:	 torch.Size([1, 2048])
ReLU output shape:	 torch.Size([1, 2048])
Dropout output shape:	 torch.Size([1, 2048])
Linear output shape:	 torch.Size([1, 2048])
ReLU output shape:	 torch.Size([1, 2048])
Dropout output shape:	 torch.Size([1, 2048])
Linear output shape:	 torch.Size([1,

In [ ]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [5]:
# 数据预处理和加载 - 纯PyTorch实现
# 为了适应AlexNet的输入尺寸(224x224),需要调整图像大小
batch_size = 128

# 定义数据转换
transform = transforms.Compose([
    transforms.Resize(224),  # 调整图像大小到224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # 归一化
])

# 加载Fashion-MNIST数据集
train_dataset = datasets.FashionMNIST(
    root='./data',  # 数据存储路径
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# 创建数据加载器
train_iter = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_iter = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f'训练集大小: {len(train_dataset)}, 测试集大小: {len(test_dataset)}')

训练集大小: 60000, 测试集大小: 10000


In [ ]:
# 训练函数 - 纯PyTorch实现
def train(net, train_iter, test_iter, num_epochs, lr, device):
    """训练模型"""
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print(f'training on {device}')
    net.to(device)
    
    # 定义优化器和损失函数
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        # 训练模式
        net.train()
        train_loss_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            
            # 前向传播
            y_hat = net(X)
            loss = loss_fn(y_hat, y)
            
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # 统计
            train_loss_sum += loss.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
            batch_count += 1
        
        # 评估模式
        test_acc = evaluate_accuracy(net, test_iter, device)
        
        print(f'epoch {epoch + 1}, loss {train_loss_sum / batch_count:.4f}, '
              f'train acc {train_acc_sum / n:.3f}, test acc {test_acc:.3f}, '
              f'time {time.time() - start:.1f} sec')

def evaluate_accuracy(net, data_iter, device):
    """评估模型准确率"""
    net.eval()
    acc_sum, n = 0.0, 0
    
    with torch.no_grad():
        for X, y in data_iter:
            X, y = X.to(device), y.to(device)
            acc_sum += (net(X).argmax(dim=1) == y).sum().item()
            n += y.shape[0]
    
    return acc_sum / n

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 训练参数
lr, num_epochs = 0.01, 10

# 开始训练
train(net, train_iter, test_iter, num_epochs, lr, device)

training on cuda
epoch 1, loss 2.3012, train acc 0.123, test acc 0.265, time 14.2 sec
epoch 2, loss 2.1908, train acc 0.271, test acc 0.547, time 14.3 sec
epoch 3, loss 0.8539, train acc 0.677, test acc 0.751, time 14.1 sec
epoch 4, loss 0.6569, train acc 0.753, test acc 0.767, time 13.9 sec
epoch 5, loss 0.5789, train acc 0.782, test acc 0.798, time 14.1 sec
epoch 6, loss 0.5285, train acc 0.801, test acc 0.815, time 14.3 sec
epoch 7, loss 0.4928, train acc 0.814, test acc 0.824, time 14.3 sec
epoch 8, loss 0.4610, train acc 0.827, test acc 0.828, time 14.0 sec
epoch 9, loss 0.4381, train acc 0.837, test acc 0.844, time 14.2 sec
epoch 10, loss 0.4168, train acc 0.844, test acc 0.847, time 14.3 sec
